## 1.import essential library and model 

In [ ]:
import cv2
from scipy import ndimage
from model import Model
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
model = Model('inference_model',use_gpu=True,use_mkldnn=False,combined=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
E0321 09:11:25.816339   219 analysis_config.cc:80] Please compile with gpu to EnableGpu()
W0321 09:11:25.816429   219 analysis_predictor.cc:1145] Deprecated. Please use CreatePredictor instead.


## 2.deal with the vedio
seperate vedio into pictures in every frame

In [ ]:
def transform_video_to_image(video_file_path, img_path):
    '''
    Save every pics of frame in video
    '''
    video_capture = cv2.VideoCapture(video_file_path)
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    count = 0
    while(True):
        ret, frame = video_capture.read() 
        if ret:
            cv2.imwrite(img_path + '%d.jpg' % count, frame)
            count += 1
        else:
            break
    video_capture.release()
    print('successfully saved, has %d pics' % count)
    return fps
fps = transform_video_to_image('baby.mp4', 'images2/')

successfully saved, has 158 pics


## 3.draw line draft

In [ ]:
from function import *

for home, dirs, files in os.walk('images2'):
        for filename in files:
            fullname = os.path.join(home, filename)
            from_mat = cv2.imread(fullname)
            width = float(from_mat.shape[1])
            height = float(from_mat.shape[0])
            new_width = 0
            new_height = 0
            if (width > height):
                from_mat = cv2.resize(from_mat, (512, int(512 / width * height)), interpolation=cv2.INTER_AREA)
                new_width = 512
                new_height = int(512 / width * height)
            else:
                from_mat = cv2.resize(from_mat, (int(512 / height * width), 512), interpolation=cv2.INTER_AREA)
                new_width = int(512 / height * width)
                new_height = 512

            from_mat = from_mat.transpose((2, 0, 1))
            light_map = np.zeros(from_mat.shape, dtype=np.float)
            for channel in range(3):
                light_map[channel] = get_light_map_single(from_mat[channel])
            light_map = normalize_pic(light_map)
            light_map = resize_img_512_3d(light_map)
            light_map = light_map.astype('float32')
            

            line_mat = model.predict(np.expand_dims(light_map, axis=0).astype('float32'))
            # 去除 batch dimension (512, 512, 3)
            line_mat = line_mat.transpose((3, 1, 2, 0))[0]
            # 裁剪 (512, 384, 3)
            line_mat = line_mat[0:int(new_height), 0:int(new_width), :]

            line_mat = np.amax(line_mat, 2)
            # denoise
            show_active_img_and_save_denoise(line_mat, './output2/' + filename)
            print('pics' + filename + 'finished')
print('all succeeded')


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/aistudio/function.py:39: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  highPass = highPass.astype(np.float)
/home/aistudio/function.py:113: DeprecationWarning: `np.float` is a deprecated alias for 

pics37.jpgfinished
pics51.jpgfinished
pics134.jpgfinished
pics20.jpgfinished
pics23.jpgfinished
pics141.jpgfinished
pics93.jpgfinished
pics142.jpgfinished
pics72.jpgfinished
pics146.jpgfinished
pics31.jpgfinished
pics96.jpgfinished
pics48.jpgfinished
pics133.jpgfinished
pics59.jpgfinished
pics3.jpgfinished
pics41.jpgfinished
pics155.jpgfinished
pics71.jpgfinished
pics60.jpgfinished
pics7.jpgfinished
pics98.jpgfinished
pics110.jpgfinished
pics66.jpgfinished


## 4.combine pics into vedio

In [ ]:
def combine_image_to_video(comb_path, output_file_path, fps=30, is_print=False):
    '''
        combine pics into vedio
    '''
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')    
    
    file_items = os.listdir(comb_path)
    file_len = len(file_items)
    # print(comb_path, file_items)
    if file_len > 0 :
        temp_img = cv2.imread(os.path.join(comb_path, file_items[0]))
        img_height, img_width = temp_img.shape[0], temp_img.shape[1]
        
        out = cv2.VideoWriter(output_file_path, fourcc, fps, (img_width, img_height))

        for i in range(file_len):
            pic_name = os.path.join(comb_path, str(i)+".jpg")
            if is_print:
                print(i+1,'/', file_len, ' ', pic_name)
            img = cv2.imread(pic_name)
            out.write(img)
        out.release()
combine_image_to_video('output2', 'work/mp4_analysis2.mp4', fps)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


## 5.combine audio

In [ ]:
def getMusic(video_name):

    video = VideoFileClip(video_name)

    return video.audio

def addMusic(video_name, audio,output_video):
    
    video = VideoFileClip(video_name)

    video = video.set_audio(audio)

    video.write_videofile(output_video)

from moviepy.editor import *
addMusic('work/mp4_analysis2.mp4',getMusic('baby.mp4'),'work/mp4_analysisnew2.mp4')

Moviepy - Building video work/mp4_analysisnew2.mp4.
MoviePy - Writing audio in mp4_analysisnew2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video work/mp4_analysisnew2.mp4



t:   0%|          | 0/159 [00:00<?, ?it/s, now=None]t:  89%|████████▊ | 141/159 [00:00<00:00, 271.87it/s, now=None]2023-03-20 16:36:08,192 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/moviepy/video/io/ffmpeg_reader.py:130: UserWarning: Warning: in file work/mp4_analysis2.mp4, 362496 bytes wanted but 0 bytes read,at frame 158/159, at time 6.97/6.97 sec. Using the last valid frame instead.
  UserWarning)

                                                               


Moviepy - Done !
Moviepy - video ready work/mp4_analysisnew2.mp4
